In [84]:
import datapackage
#import gensim
import pandas as pd
import re

package=datapackage.Package('http://next.obudget.org/datapackages/procurement/spending/latest-contract-spending/datapackage.json')
response = package.resources[0]
iterator = response.iter(keyed=True)

df=pd.DataFrame()

items = []
counter = 0
for row in iterator:
    for column_name in list(row.keys()):
        if column_name not in df.columns:
            df.insert(column=column_name,loc=len(df.columns),value=None)
    items.append(row)
    counter+=1
    if counter % 10000 == 0:
        print("downloaded: ",counter," items")
print("downloaded: ",counter," items")

df = pd.DataFrame(items)

# add a row index to df
df['doc_index'] = df.index    


downloaded:  10000  items
downloaded:  20000  items
downloaded:  30000  items
downloaded:  40000  items
downloaded:  50000  items
downloaded:  60000  items
downloaded:  70000  items
downloaded:  80000  items
downloaded:  90000  items
downloaded:  100000  items
downloaded:  110000  items
downloaded:  120000  items
downloaded:  130000  items
downloaded:  140000  items
downloaded:  150000  items
downloaded:  160000  items
downloaded:  170000  items
downloaded:  180000  items
downloaded:  190000  items
downloaded:  200000  items
downloaded:  210000  items
downloaded:  220000  items
downloaded:  230000  items
downloaded:  240000  items
downloaded:  250000  items
downloaded:  260000  items
downloaded:  270000  items
downloaded:  280000  items
downloaded:  290000  items
downloaded:  300000  items
downloaded:  310000  items
downloaded:  320000  items
downloaded:  330000  items
downloaded:  340000  items
downloaded:  350000  items
downloaded:  360000  items
downloaded:  370000  items
downloaded

In [154]:
# tokenize verbal column, add meta information without modifying it
def tokenize(row):
    # add here the columns you would like to index
    columns_list = ['explanation','purpose'] 
    meta_columns_list = ['budget_code','budget_title','buyer_description','exemption_reason','publisher','purchase_method','purchasing_unit','supplier_name']

    tokenized = []
    for column_name in columns_list:
        tokenized+=tokenize_text(row[column_name])
    for meta_column in meta_columns_list:
        tokenized+=[meta_column+": "+str(row[meta_column])]
    return ", ".join(tokenized)


def tokenize_text(text):    # tokenie textual documents
                            # get rid of non alpha numberics, split to words in selected columns
    regxlst = [re.compile(x) for x in [r'\d+',]]    # regular expressions list for clean ups: 
                                                             # 1. only digits    
    stringed_text = str(text)
    ignore_signs = ["\,","\:","\;","\.","\&","\$","\-","\=","\(","\)","\d+","\\n"]
    cleaned_1 = re.sub("|".join(ignore_signs),"",stringed_text) # remove non-alphanumberic characters
    cleaned_2 = re.sub("  "," ",cleaned_1)                # no more double spaces
    cleaned_3 = cleaned_2.split()                       # split into separate words list
    cleaned_4 =  [word for regex in regxlst for word in cleaned_3 if not(regex.match(word))] # filter by regular expressions
    cleaned_5 = [word for word in cleaned_4 if word is not None]
    return cleaned_5

# creata words list per doc, add to Dataframe ('tokenized')

df['tokenized'] = df.apply(tokenize,axis=1)

In [155]:
import gensim
import numpy
gen_docs = [[ word for word in doc.split(", ") if (word!= "" and word!='None')] for doc in df['tokenized']]
dictionary = gensim.corpora.Dictionary(gen_docs)

print("Number of words in dictionary:",len(dictionary))

corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)

Number of words in dictionary: 110783
TfidfModel(num_docs=422342, num_nnz=5331017)


In [156]:
sims = gensim.similarities.Similarity('',
                                      tf_idf[corpus],
                                      num_features=len(dictionary))

In [192]:
def find_similiar(query_index): # choose a Dataframe row index to query
    query_doc = tokenize(df.iloc[query_index]).split(", ")
    # print(query_doc)
    query_doc_bow = dictionary.doc2bow(query_doc)
    # print(query_doc_bow)
    query_doc_tf_idf = tf_idf[query_doc_bow]
    # print(query_doc_tf_idf)

    results = sims[query_doc_tf_idf]
    results = sorted(enumerate(results), key=lambda item: -item[1])[1:10]
    print("results: ",results)
    
    return results

In [209]:
query_index = 166825
results = find_similiar(query_index)

print("query: ")
print(df.iloc[query_index])
print(" ---- similiar: ---- ")
print(df.iloc[results[3][0]])

results:  [(166825, 1.0), (34723, 0.5619016), (34724, 0.5619016), (216097, 0.51524925), (216098, 0.51524925), (33079, 0.5090351), (33080, 0.5090351), (42444, 0.4872114), (42445, 0.4872114)]
query: 
amount_contracts                                                        2632.5
budget_code                                                         0007600230
budget_title                                                         מחוז צפון
buyer_description                                                  [מחוז צפון]
company_id                                                           514449370
currency                                                                   ILS
end_date                                                                  None
entity_id                                                            514449370
entity_kind                                                            company
entity_name                                        ש.ה. גורן ניהול ויזמות בע"מ
executed    